In [5]:
import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')

import requests
import pandas as pd
import govcxanalyzer.data_preprocessing as dp
from govcxanalyzer import eda, lda
from govcxanalyzer import utils
import govcxanalyzer.text_analysis as ta
import numpy as np

tp = utils.get_date_timestamp()

print(tp)
import time


mergedcxdata =pd.read_csv("../data/mergedcxdata-2023-05-21.csv")
mergedcxdata = mergedcxdata.rename(columns={col: col.replace("attributes_", "") for col in list(mergedcxdata.columns)})

idcols = [col for col in mergedcxdata.columns if col.endswith("id")]
dp.make_categorical_subset(mergedcxdata).sample(2)



2023-08-29_22:35:15
Index(['type', 'organization_name', 'organization_abbreviation',
       'service_provider_name', 'collection_name', 'service_provided',
       'transaction_point', 'channel', 'omb_control_number',
       'federal_register_url', 'q1_text', 'q2_text', 'q3_text', 'q4_text',
       'q5_text', 'q6_text', 'q7_text', 'q8_text', 'q9_text', 'q10_text',
       'q11_text', 'created_at', 'updated_at', 'operational_metrics',
       'service_name', 'service_slug', 'service_desc',
       'service_provider_desc'],
      dtype='object')


,type,organization_name,organization_abbreviation,service_provider_name,collection_name,service_provided,transaction_point,channel,omb_control_number,federal_register_url,...,q9_text,q10_text,q11_text,created_at,updated_at,operational_metrics,service_name,service_slug,service_desc,service_provider_desc
223,omb_cx_reporting_collections,Department of State,DOS,Passport Services,CX Quarterly Reporting,Passport information and status on pending pas...,Immediately following a phone call with a cust...,automated_phone,1090-0007,NaN,...,NaN,NaN,NaN,2022-01-19T22:57:17.933Z,2022-02-04T18:54:45.754Z,Contact Centers - Call answer rate of 75%.,Passport Services,dos-ca,Passport Services is a directorate within the ...,Passport Services is a directorate within the ...
103,omb_cx_reporting_collections,Department of Veterans Affairs,VA,Veterans Benefits Administration,CX Quarterly Reporting,AMA Supplemental Claim,e-mail survey with online response collection,email,2900-0770,https://www.reginfo.gov/public/do/PRAViewIC?re...,...,NaN,NaN,NaN,2021-07-01T05:06:41.773Z,2021-08-04T19:40:35.432Z,NaN,Contacting the Benefits Administration contact...,va-vba,VBA sends surveys to those who interact with i...,The Veterans Benefits Administration (VBA) ser...


In [6]:
## read in example hisp fake data


example_survey_df =pd.read_csv("../data/fake-omb-a11-survey-response-level-data.csv")

example_survey_df.sample(2)
example_survey_df["FreeText"].dropna()

0      I was able to easily find what I was looking for.
1                   The information was not very useful.
2      I had hoped for a more robust sample reports s...
3                the staff is so helpful...thank you !!!
4                                             lova yall!
                             ...                        
108          one of the worse services i have every used
109                                not a nice experience
110                                very rude interaction
111                   Excellent website! consider this..
112                                     Its nice website
Name: FreeText, Length: 99, dtype: object

In [8]:
import warnings
warnings.filterwarnings('ignore')

vis = lda.create_vis_lda_from_df_textcol(df=example_survey_df, textcol='FreeText')


In [10]:
import pyLDAvis
pyLDAvis.enable_notebook()

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.227292 -0.044518       1        1  13.965082
11    -0.100326 -0.181383       2        1  10.680956
2     -0.044768  0.126118       3        1   9.455000
12    -0.032012  0.058311       4        1   8.772333
10    -0.003381 -0.018485       5        1   7.316023
1     -0.014920  0.018998       6        1   7.189554
8     -0.011457  0.013738       7        1   7.090185
6     -0.009174  0.010954       8        1   6.504426
7     -0.004009  0.005007       9        1   5.296414
9     -0.003171  0.004129      10        1   4.754406
14    -0.002381  0.003323      11        1   4.688915
0     -0.006307  0.006170      12        1   4.526635
5     -0.000297  0.000115      13        1   3.674915
4      0.001603 -0.000480      14        1   3.562000
3      0.003308 -0.001999      15        1   2.523155, topic_info=              Term      Freq     Total Category  logprob  loglift
28      easi_navig  1.000000  1.000000  Default  30.0000  30.0000
75    process_good  1.000000  1.000000  Default  29.0000  29.0000
68       work_hisp  1.000000  1.000000  Default  28.0000  28.0000
48      keep_chang  0.000000  0.000000  Default  27.0000  27.0000
39      even_finsh  0.000000  0.000000  Default  26.0000  26.0000
..             ...       ...       ...      ...      ...      ...
106  much_transpar  0.019895  0.912053  Topic15  -6.1484  -0.1456
108  feedback_team  0.019895  1.124161  Topic15  -6.1484  -0.3547
50     inform_easi  0.019895  1.068143  Topic15  -6.1484  -0.3036
62    logist_field  0.019895  1.051736  Topic15  -6.1484  -0.2881
112   commun_group  0.019895  1.124183  Topic15  -6.1484  -0.3547

[582 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
4         1  0.889519     .great_websit
0         3  0.950808        abl_easili
346       4  0.963528     admiss_approv
307       7  0.974043  after-hr_support
266       6  1.000186      agenc_commit
...     ...       ...               ...
68        5  0.712804         work_hisp
357       6  1.000186       wors_servic
98        8  1.009946        would_nice
60        8  1.009929           ye_love
335       2  0.936187      year_respons

[230 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 12, 3, 13, 11, 2, 9, 7, 8, 10, 15, 1, 6, 5, 4])

{'mdsDat': {'x': [0.22729223812821156,
   -0.10032631350990429,
   -0.04476787600871834,
   -0.032011948976667634,
   -0.0033812873334625746,
   -0.014920076850612635,
   -0.011456851778701755,
   -0.00917383224864457,
   -0.004009173945453552,
   -0.0031709565427289785,
   -0.0023814867734007735,
   -0.006307161759243523,
   -0.00029678772942134127,
   0.0016030992658902796,
   0.003308416062858178],
  'y': [-0.04451760705592148,
   -0.18138267122174742,
   0.12611812799025535,
   0.05831126966400822,
   -0.01848545909466883,
   0.018998122508268264,
   0.013738360341329027,
   0.010954417225990205,
   0.00500678292023803,
   0.00412918073352172,
   0.0033230216007970365,
   0.0061696081797978745,
   0.00011532803562668361,
   -0.0004797328836479039,
   -0.001998748943847572],
  'topics': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
  'cluster': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'Freq': [13.965081878271887,
   10.680956397355033,
   9.454999503138309,
   8.77233

In [ ]:
import pyLDAvis

pyLDAvis.enable_notebook()



In [ ]:
### Now let's pull some real customer feedback
import requests
from bs4 import BeautifulSoup
# pulling reviews
# obtaining data


base_url = "https://www.sitejabber.com/reviews/"

urls2get = ["usajobs.gov", "cdc.gov","ed.gov", "treasurydirect.gov"]

def get_soup_from_revies(url_):
    
    reviews = requests.get(url_)
    
    soup = BeautifulSoup(reviews.text)
    return soup

    
soupl = []

for ur_ in urls2get:
    soup = get_soup_from_revies(f"{base_url}{ur_}")
    
    soupl.append(soup)


